In [1]:
!pip install google-cloud-aiplatform --upgrade --user --quiet

In [ ]:
!pip install datasets

In [2]:
import pandas as pd
from IPython.display import Markdown, display
from vertexai.language_models import TextGenerationModel

2023-09-14 15:25:57.179687: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-14 15:25:58.247257: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-09-14 15:25:58.247384: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [3]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

In [12]:
def query_model(prompt, model_predictor):    
    response = model_predictor.predict(
        prompt=prompt
    )
    
    return {'output': response.text} 

In [1]:
import datasets

eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")["eval"]

In [4]:
eval_set[0]

{'instruction': 'What are the names of some famous actors that started their careers on Broadway?',
 'output': 'Some famous actors that started their careers on Broadway include: \n1. Hugh Jackman \n2. Meryl Streep \n3. Denzel Washington \n4. Julia Roberts \n5. Christopher Walken \n6. Anthony Rapp \n7. Audra McDonald \n8. Nathan Lane \n9. Sarah Jessica Parker \n10. Lin-Manuel Miranda',
 'generator': 'text_davinci_003',
 'dataset': 'helpful_base'}

In [8]:
len(eval_set)

805

In [19]:
import time

# Create an empty list to store the results
results_list = []

for i in range(len(eval_set)):
    example = eval_set[i]
    result = query_model(prompt = example["instruction"], model_predictor = generation_model)
    results_list.append(result)
    if i % 60 == 0:
        time.sleep(60)
    
# Create a new DataFrame from the results list
results_list = pd.DataFrame(results_list)

results_list

,output
0,* Meryl Streep\n* Marlon Brando\n* James Dean\...
1,The names of the 50 US states come from a vari...
2,Kickball is a game that is played with a ball ...
3,"* ""The Charleston"" by James P. Johnson\n* ""Ain..."
4,1. **Choose the right wrapping paper.** The si...
...,...
800,**Intro**\n\nJazz is a music genre that origin...
801,**A Trip to Hawaii: The Cultural Experience**\...
802,"The new science fiction film ""Interstellar"" is..."
803,**Intro**\n\nWelcome to the first episode of o...


In [20]:
results_list.to_json('palm_alpaca_eval.json')

In [21]:
palm_alpaca_eval_df = pd.read_json('palm_alpaca_eval.json')
palm_alpaca_eval_df.head()

,output
0,* Meryl Streep\n* Marlon Brando\n* James Dean\...
1,The names of the 50 US states come from a vari...
10,Retinoids are a class of drugs that are used t...
100,"Ingredients:\n\n* 1 pound (450g) dried tofu, c..."
101,* A zombie: This is a classic Halloween costum...


In [27]:
len(palm_alpaca_eval_df)

805

In [22]:
def remove_newlines(text):
    cleaned_text = text.replace('\n', ' ')
    return cleaned_text

In [23]:
palm_alpaca_eval_df['output_v2'] = palm_alpaca_eval_df['output'].apply(remove_newlines)

In [39]:
palm_alpaca_eval_df.shape

(805, 2)

In [40]:
eval_set_df = eval_set.to_pandas()

In [41]:
palm_alpaca_eval_df.rename(columns={'output': 'palm_output', 'output_v2': 'palm_output_v2'}, inplace=True)

In [42]:
combined_df = pd.concat([eval_set_df, palm_alpaca_eval_df,], axis=1)
combined_df.head()

,instruction,output,generator,dataset,palm_output,palm_output_v2
0,What are the names of some famous actors that ...,Some famous actors that started their careers ...,text_davinci_003,helpful_base,* Meryl Streep\n* Marlon Brando\n* James Dean\...,* Meryl Streep * Marlon Brando * James Dean * ...
1,How did US states get their names?,US states get their names from a variety of so...,text_davinci_003,helpful_base,The names of the 50 US states come from a vari...,The names of the 50 US states come from a vari...
2,"Hi, my sister and her girlfriends want me to p...","Kickball is a game similar to baseball, but wi...",text_davinci_003,helpful_base,Kickball is a game that is played with a ball ...,Kickball is a game that is played with a ball ...
3,What is some cool music from the 1920s?,Some cool music from the 1920s includes jazz c...,text_davinci_003,helpful_base,"* ""The Charleston"" by James P. Johnson\n* ""Ain...","* ""The Charleston"" by James P. Johnson * ""Ain'..."
4,How do I wrap a present neatly?,1. Start by gathering the supplies you will ne...,text_davinci_003,helpful_base,1. **Choose the right wrapping paper.** The si...,1. **Choose the right wrapping paper.** The si...


In [43]:
temp_df = combined_df[['instruction', 'output']]
temp_df.to_json("combined_df.json", orient='records')

In [6]:
import pandas as pd

combined_df = pd.read_json("combined_df.json")
combined_df.head()

,instruction,output
0,What are the names of some famous actors that ...,Some famous actors that started their careers ...
1,How did US states get their names?,US states get their names from a variety of so...
2,"Hi, my sister and her girlfriends want me to p...","Kickball is a game similar to baseball, but wi..."
3,What is some cool music from the 1920s?,Some cool music from the 1920s includes jazz c...
4,How do I wrap a present neatly?,1. Start by gathering the supplies you will ne...


In [7]:
pd.set_option('display.max_colwidth', None)
combined_df.head(100)

,instruction,output
0,What are the names of some famous actors that started their careers on Broadway?,Some famous actors that started their careers on Broadway include: \n1. Hugh Jackman \n2. Meryl Streep \n3. Denzel Washington \n4. Julia Roberts \n5. Christopher Walken \n6. Anthony Rapp \n7. Audra McDonald \n8. Nathan Lane \n9. Sarah Jessica Parker \n10. Lin-Manuel Miranda
1,How did US states get their names?,"US states get their names from a variety of sources, including Native American tribes, Spanish explorers, British colonists, and even presidents. For example, the state of Alabama was named after the Native American tribe that lived in the area, while the state of Florida gets its name from the Spanish explorer, Ponce de Leon, who explored the area in the 1500s. Other states are named after English kings (like Virginia, named after England's ""Virgin Queen,"" Queen Elizabeth I) or presidents (like Washington, named after George Washington)."
2,"Hi, my sister and her girlfriends want me to play kickball with them. Can you explain how the game is played, so they don't take advantage of me?","Kickball is a game similar to baseball, but with a large rubber ball instead of a bat and a ball. The game is usually played with two teams of six players each. Each team has three bases and a home plate. The players on the kicking team line up at home plate and take turns kicking the ball. The object of the game is to score runs by running around all three bases and back to home plate without being tagged out by the defense. The team with the most runs at the end of the game is the winner."
3,What is some cool music from the 1920s?,"Some cool music from the 1920s includes jazz classics like Duke Ellington's ""Mood Indigo,"" Louis Armstrong's ""West End Blues,"" and Bessie Smith's ""Down Hearted Blues."" Other popular tunes from the era include Fats Waller's ""Ain't Misbehavin,"" George Gershwin's ""Rhapsody in Blue,"" and Irving Berlin's ""Puttin' On the Ritz."""
4,How do I wrap a present neatly?,"1. Start by gathering the supplies you will need to wrap the present, such as wrapping paper, tape, scissors, and ribbon.\n\n2. Cut the wrapping paper to the size of the gift, ensuring that there is enough paper to cover the entire gift.\n\n3. Place the gift in the center of the wrapping paper, and fold the sides in, tucking the excess paper at the bottom.\n\n4. Secure the folded paper with tape.\n\n5. Cut a length of ribbon and tie it around the gift.\n\n6. Trim the excess ribbon with scissors.\n\n7. Add a decorative bow or other embellishment to the top of the gift."
...,...,...
95,I want to start saving some money by growing my own food. Can I do this during the winter with an indoor garden?,"Yes, you can grow your own food indoors during the winter with an indoor garden. An indoor garden can be set up with a hydroponic system, or with containers, shelves, and lighting. You will need soil, seeds, and some basic gardening tools to get started. Additionally, you may need to purchase grow lights if you don't have enough natural sunlight coming in."
96,How can I spice things up in the bedroom?,1. Try new positions. \n2. Introduce toys and props. \n3. Use massage and body oils. \n4. Play music and create a mood. \n5. Bring food into the bedroom. \n6. Talk dirty. \n7. Take pictures of each other.\n8. Role play. \n9. Make a game of it. \n10. Try mutual masturbation.
97,Is human blood ever any color other than red?,"No, human blood is always red."
98,Please tell me about pearls. Where are they found?,"Pearls are a type of gemstone that are produced by mollusks, most often oysters. They are formed when a grain of sand or other irritant gets inside the mollusk and the mollusk secretes layers of calcium carbonate around it in order to protect itself. The result is a beautiful gemstone with a unique iridescence. Pearls are found all over the world, but the most valuable pearls come from the South Pacific and the Persian Gulf."
